## Extract patch images from patch annotations
Read XML label file and extract patch images from original image

In [ ]:
import cv2
import os
import json
import xml.etree.ElementTree as ET

home = '/media/data1/Ace/11000_Marine_objects/11200_Dataset/11220_Images/11221_KOMPSAT-5'
img_home = os.path.join(home, 'PNG', 'tmp')
ann_home = os.path.join(home, 'Annotations')
save_home = os.path.join(home, 'PNG', 'patch')
os.makedirs(save_home, exist_ok=True)

empty_img = []
num_patch = 0

def check_box(box, size=1024):
    """Check box size
    
    Check if box size is the same as "size"
    Args:
        box(list[int,]): bounding box. [xmin, ymin, xmax, ymax]
        size(int): size of bouding box. default: 1024
        
    Returns:
        (bool): True if box size is the same as "size"
    """
    width = box[2] - box[0]
    height = box[3] - box[1]
    
    if width == size and height == size:
        return True
    
    return False

def correct_box(box, size=1024):
    """Correct box coordinates
    
    In labelImg, if xmin/ymin is 0, it record 0 as 1.
    
    For example, if the size of a box is 1024,
    [0, 0, 1024, 1024] -> [1, 1, 1024, 1024]
    [1, 1, 1025, 1025] -> [1, 1, 1025, 1025]
    [2, 2, 1026, 1026] -> [2, 2, 1026, 1026]
    
    So, you have to change the coordinate of the box of a size "1023".
    For example,
    [1, 1, 1024, 1024] -> [0, 0, 1024, 1024] (Change This)
    [1, 1, 1025, 1025] -> [1, 1, 1025, 1025]
    [2, 2, 1026, 1026] -> [2, 2, 1026, 1026]
    
    Sometimes, for some reason, when the box is at the corner, its size become (size-1).
    You have to check this case and correct it.
    For example,
    [1, 1, 1023, 1023] -> [1, 1, 1024, 1024]
    
    Args:
        box(list[int,]): bounding box. [xmin, ymin, xmax, ymax]
        size(int): size of bouding box. default: 1024
    Returns:
        box(list[int,]): bounding box. [xmin, ymin, xmax, ymax]
    """
    width = box[2] - box[0]
    height = box[3] - box[1]
    
    if width != size:
        if box[0] > 1:
            box[2] = box[0] + size
        else:
            box[0] = box[2] - size
        
    if height != size:
        if box[1] > 1:
            box[3] = box[1] + size
        else:
            box[1] = box[3] - size
        
    return box
    

for img_name in os.listdir(img_home):
    print('Processing {}'.format(img_name))
    img_path = os.path.join(img_home, img_name)
    ann_path = os.path.join(ann_home, img_name[:-3] + 'xml')
    
    tree = ET.parse(ann_path)
    root = tree.getroot()
    objs = root.findall('object')
    if objs:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        for obj in objs:
            # Coord of labelImg starts from 1
            bndbox = [int(coord.text) for coord in obj.find('bndbox').getchildren()]  # bndbox = [xmin, ymin, xmax, ymax]
            bndbox = correct_box(bndbox)            
            
            if check_box(bndbox):
                patch_img = img[bndbox[1]:bndbox[3], bndbox[0]:bndbox[2]]
                patch_name = '{}_{}_{}_{}_{}.png'.format(img_name[:-4], bndbox[0], bndbox[1], bndbox[2]-1, bndbox[3]-1)
                cv2.imwrite(os.path.join(save_home, patch_name), patch_img)
                num_patch += 1
            else:
                print('Wrong boxes: ', bndbox)
    else:
        empty_img.append(img_name)
        
if empty_img:
    print('Annotations not eixst: ', empty_img)
print('Total number of patches: ', num_patch)
print('Done')

Processing K5_20211107231325_000010_45098_A_ES11_HH_SCS_B_L1A.png
